In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

In [2]:
header_list = ["eth-usd", "price", "volume", "timestamp"]
data_2 = pd.read_csv("eth.usd.2018.csv", names = header_list)
data_2 = data_2.truncate(after = 100000)

In [356]:
type(data_2.head(1))

pandas.core.frame.DataFrame

In [3]:
import keras
model = keras.models.load_model("DNN_model")

Using TensorFlow backend.






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [4]:
model

In [5]:
import pickle
params = pickle.load(open("params.pkl", "rb"))
mu = params["mu"]
std = params['std']

In [6]:
mu

price        7.374525e+02
volume       1.294626e+00
timestamp    1.514778e+09
returns      2.208643e-07
dir          4.020547e-01
sma          8.350337e-03
boll                  inf
min         -7.912891e-03
max          8.473131e-03
mom          2.214071e-07
vol          5.059674e-03
dir_lag1     4.020698e-01
dir_lag2     4.020698e-01
dir_lag3     4.020850e-01
dir_lag4     4.020850e-01
dir_lag5     4.020850e-01
sma_lag1     8.366884e-03
sma_lag2     8.382895e-03
sma_lag3     8.399518e-03
sma_lag4     8.416712e-03
sma_lag5     8.434746e-03
min_lag1    -7.912716e-03
min_lag2    -7.912542e-03
min_lag3    -7.912367e-03
min_lag4    -7.912170e-03
min_lag5    -7.911943e-03
max_lag1     8.473106e-03
max_lag2     8.473082e-03
max_lag3     8.473059e-03
max_lag4     8.473057e-03
max_lag5     8.473088e-03
mom_lag1     2.289258e-07
mom_lag2     2.469307e-07
mom_lag3     2.648616e-07
mom_lag4     3.154633e-07
mom_lag5     3.565515e-07
vol_lag1     5.059628e-03
vol_lag2     5.059586e-03
vol_lag3    

In [7]:
std

price           7.764555
volume          7.599794
timestamp    7145.471525
returns         0.005315
dir             0.490317
sma             0.743618
boll                 NaN
min             0.005312
max             0.005891
mom             0.001938
vol             0.001790
dir_lag1        0.490320
dir_lag2        0.490320
dir_lag3        0.490323
dir_lag4        0.490323
dir_lag5        0.490323
sma_lag1        0.743613
sma_lag2        0.743608
sma_lag3        0.743602
sma_lag4        0.743597
sma_lag5        0.743592
min_lag1        0.005312
min_lag2        0.005312
min_lag3        0.005312
min_lag4        0.005312
min_lag5        0.005312
max_lag1        0.005891
max_lag2        0.005891
max_lag3        0.005891
max_lag4        0.005891
max_lag5        0.005891
mom_lag1        0.001938
mom_lag2        0.001938
mom_lag3        0.001938
mom_lag4        0.001938
mom_lag5        0.001938
vol_lag1        0.001790
vol_lag2        0.001790
vol_lag3        0.001790
vol_lag4        0.001790


In [474]:
class Predict:
    def __init__(self, instrument, window, lags, model, mu, std, start_time=0, user_input=None):
        self.user_input = user_input
        self.start_time = start_time
        self.instrument = instrument
        self.window = window
        self.lags = lags
        self.model = model
        self.mu = mu
        self.std = std
        self.raw_data = None            # review later
        #self.tick_data = pd.Dataframe() # review later
        self.remaining_fund = 1000000
        self.eth_position = 0
        self.btc_position = 0

    def define_strategy(self):
        df = data_2.copy()
        df = df.append(self.user_input, ignore_index=True) # append the latest tick
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df['dir'] = np.where(df['returns'] > 0,1,0)
        df["sma"] = df[self.instrument].rolling(self.window).mean() - df[self.instrument].rolling(150).mean()
        df["min"] = df[self.instrument].rolling(self.window).min() / df[self.instrument] - 1
        df["max"] = df[self.instrument].rolling(self.window).max() / df[self.instrument] - 1
        df["mom"] = df["returns"].rolling(3).mean()
        df["vol"] = df["returns"].rolling(self.window).std()
        df.dropna(inplace = True)
        

        # create logs
        self.cols = []
        features = ["dir", "sma", "min", "max", "mom", "vol"]

        for f in features:
            for lag in range(1, self.lags +1):
                col = "{}_lag_{}".format(f, lag)
                df[col] = df[f].shift(lag)
                self.cols.append(col)
        df.dropna(inplace=True)

        # standardization
        self.mu = df.mean()
        self.std = df.std()
        df_s = (df - self.mu) / self.std
        # predict
        df["proba"] = self.model.predict(df_s[self.cols])

        # Determine position
        #df = df.loc[self.start_time:].copy() # remove all historical data
        df["position"] = np.where(df.proba < 0.47, 1, np.nan)
        df["position"] = np.where(df.proba > 0.53, 1, df.position)
        df["position"] = df.position.ffill().fillna(0)

        self.data = df.copy()

    def add_time(self, time):
        self.start_time = time

    def add_input(self, user_input):
        self.user_input = user_input

    def get_recent_df_row(self):
        self.define_strategy()
        return self.data['proba'].iloc[-1:]

    def update_up(self, up1, up2, up3, up4):
        self.up_1 = up1
        self.up_2 = up2
        self.up_3 = up3
        self.up_4 = up4


In [475]:
predict = Predict(instrument='price', window=50, lags=5, model=model, mu=mu, std=std)

In [476]:
from typing import Any
from io import StringIO

In [480]:
def algorithm(csv_row: str):
    """ Trading Algorithm

    Add your logic to this function. This function will simulate a streaming
    interface with exchange trade data. This function will be called for each
    data row received from the stream.

    The context object will persist between iterations of your algorithm.

    Args:
        csv_row (str): one exchange trade (format: "exchange pair", "price", "volume", "timestamp")
        context (dict[str, Any]): a context that will survive each iteration of the algorithm

    Generator:
        response (dict): "Fill"-type object with information for the current and unfilled trades
    
    Yield (None | Trade | [Trade]): a trade order/s; None indicates no trade action
    """
    # algorithm logic...
    # eth-usd,172,1,1511023123890
    #StringData = StringIO("""
    #eth-usd,price,volume,timestamp
    #""" + csv_row)
    #csv_row = pd.read_csv(StringData, sep=',')
    #start_time = csv_row.iloc[:,3]
    lst = [csv_row.split(",")] # lst = ['eth-usd','300','1','1265667778']
    lst[0][1] = float(lst[0][1])
    lst[0][2] = float(lst[0][2])
    lst[0][3] = float(lst[0][3])
    csv_row = pd.DataFrame(lst, columns=['pair','price', 'volume','timestamp'])
    #predict.add_time(start_time)
    predict.add_input(csv_row) # list: ["eth-usd","172","1","151519237548023"]
    probability = predict.get_recent_df_row()
    ticker = lst[0][0][5:8]

    #print(probability)
    if probability > 0.53:
        if predict.remaining_fund >= lst[0][1]:
            response = yield Trade("BUY", ticker, Decimal(1))
            if ticker == 'btc'
                predict.btc_position += 1
            else:
                predict.eth_position += 1
        else:
            response = yield None
    elif probability < 0.47:
        if ticker == 'btc':
            if predict.btc_position >= 1:
                response = yield Trade("SELL", ticker, Decimal(1))
            else:
                response = yield None
        if ticker == 'eth':
            if predict.eth_position >= 1:
                response = yield Trade("SELL", ticker, Decimal(1))
            else:
                response = yield None
    else:
        response = yield None
    #else:
    #    response = "None"
    
    #print(response)

    
    #response = yield None # example: Trade(BUY, 'xbt', Decimal(1))

    # algorithm clean-up/error handling...

In [481]:
print(algorithm("okfq-xbt-usd,14682.26,2,1514765115"))

Series([], Name: proba, dtype: float64)
xbt
None


In [438]:
data_2.dtypes

eth-usd       object
price        float64
volume       float64
timestamp    float64
dtype: object